In [2]:
# Load cleaned silver delta table



injection_lh_fpath = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/staging_lakehouse_sw.Lakehouse/Files"
bronz_lh_fpath = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/bronze_lakehouse_sw.Lakehouse/Files"
silver_lh_fpath = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/silver_lakehouse_sw.Lakehouse/Files"
gold_lh_tpath = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/gold_lakehouse_sw1.Lakehouse/Tables"
silver_delta_table=r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/silver_lakehouse_1.Lakehouse/Tables"


hr_data_silver = spark.read.format("delta").load(silver_delta_table + "//" + "hr_data_silver")
hr_data_silver.printSchema()
hr_data_silver.show(5) 

StatementMeta(, 7d6c9461-c616-465c-b378-855d848a7d2c, 5, Finished, Available, Finished)

root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- race: string (nullable = true)
 |-- department: string (nullable = true)
 |-- jobtitle: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- termdate: timestamp (nullable = true)
 |-- location_city: string (nullable = true)
 |-- location_state: string (nullable = true)
 |-- employee_id: integer (nullable = true)



+----------+----------+---------+----------+--------------+--------------------+--------------------+--------------------+--------+----------+-------------------+-------------+--------------+-----------+
|        id|first_name|last_name| birthdate|        gender|                race|          department|            jobtitle|location| hire_date|           termdate|location_city|location_state|employee_id|
+----------+----------+---------+----------+--------------+--------------------+--------------------+--------------------+--------+----------+-------------------+-------------+--------------+-----------+
|70-0828882|  Thaddeus|  Eldered|1972-11-14|Non-Conforming|American Indian o...|          Accounting|       Accountant IV|  Remote|2019-06-20|               NULL|    Ann Arbor|      Michigan|  700828882|
|53-8809496|     Corny|  Heymann|2000-02-19|Non-Conforming|Native Hawaiian o...|  Product Management|     Project Manager|  Remote|2005-10-15|               NULL|       Peoria|      Il

In [ ]:
from pyspark.sql.functions import col, floor, datediff, current_date

StatementMeta(, 7d6c9461-c616-465c-b378-855d848a7d2c, -1, Cancelled, , Cancelled)

In [ ]:
hr_data_silver.filter(col("birthdate").isNull()).count()

StatementMeta(, 7d6c9461-c616-465c-b378-855d848a7d2c, 7, Finished, Available, Finished)

0

In [ ]:

dim_department = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/gold_lakehouse_sw1.Lakehouse/Tables/DimDepartment" )
dim_location = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/gold_lakehouse_sw1.Lakehouse/Tables/DimLocation")

# Join to get department_id and location_id
joined_df = hr_data_silver \
    .join(dim_department, on="department", how="left") \
    .join(dim_location, on=["location_city", "location_state"], how="left")


StatementMeta(, 7d6c9461-c616-465c-b378-855d848a7d2c, 8, Finished, Available, Finished)

In [ ]:
from pyspark.sql.functions import col, count, avg, sum, when, round, concat_ws, concat, lit


from pyspark.sql.functions import col, floor, datediff, current_date

joined_df = joined_df.withColumn(
    "age",
    floor(datediff(current_date(), col("birthdate")) / 365.25)
)


fact_df = joined_df.groupBy("department_id", "location_id").agg(
    count("employee_id").alias("total_employees"),
    round(avg("age"), 1).alias("avg_age"),

    concat_ws(", ",
        concat(lit("Male: "), round(100 * sum(when(col("gender") == "Male", 1).otherwise(0)) / count("*"), 1), lit("%")),
        concat(lit("Female: "), round(100 * sum(when(col("gender") == "Female", 1).otherwise(0)) / count("*"), 1), lit("%")),
        concat(lit("Other: "), round(100 * sum(when(col("gender") == "Other", 1).otherwise(0)) / count("*"), 1), lit("%"))
    ).alias("gender_distribution"),

    round(100 * sum(when(col("termdate").isNotNull(), 1).otherwise(0)) / count("employee_id"), 2).alias("turnover_rate")
)


StatementMeta(, 7d6c9461-c616-465c-b378-855d848a7d2c, 9, Finished, Available, Finished)

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

fact_df = fact_df.withColumn("fact_id", monotonically_increasing_id())
fact_df.printSchema()

StatementMeta(, 7d6c9461-c616-465c-b378-855d848a7d2c, 10, Finished, Available, Finished)

root
 |-- department_id: integer (nullable = true)
 |-- location_id: integer (nullable = true)
 |-- total_employees: long (nullable = false)
 |-- avg_age: double (nullable = true)
 |-- gender_distribution: string (nullable = false)
 |-- turnover_rate: double (nullable = true)
 |-- fact_id: long (nullable = false)



In [ ]:
from pyspark.sql.functions import expr, when, col, count, avg, round, sum, lower, trim, monotonically_increasing_id

# Clean gender values
joined_df = joined_df.withColumn("gender", lower(trim(col("gender"))))

# Aggregation logic
fact_df = joined_df.groupBy("department_id", "location_id").agg(
    count("*").alias("total_employees"),
    round(avg("age"), 1).alias("avg_age"),
    
    expr("""
     concat_ws(', ',
        concat('Male: ', ROUND(100 * sum(CASE WHEN gender = 'm' THEN 1 ELSE 0 END) / count(*), 1), '%'),
        concat('Female: ', ROUND(100 * sum(CASE WHEN gender = 'fm' THEN 1 ELSE 0 END) / count(*), 1), '%'),
        concat('Other: ', ROUND(100 * sum(CASE WHEN gender = 'non-conforming' THEN 1 ELSE 0 END) / count(*), 1), '%')
    )
    """).alias("gender_distribution"),

    round(100 * sum(when(col("termdate").isNotNull(), 1).otherwise(0)) / count("*"), 2).alias("turnover_rate")
)

# Add IDs
fact_df = fact_df.withColumn("fact_id", monotonically_increasing_id() + 1)
fact_df = fact_df.withColumn("employee_id", monotonically_increasing_id() + 101)

# Final column order
fact_df_final = fact_df.select(
    "fact_id",
    "employee_id",
    "location_id",
    "department_id",
    "total_employees",
    "avg_age",
    "gender_distribution",
    "turnover_rate"
)

display(fact_df_final)


StatementMeta(, 7d6c9461-c616-465c-b378-855d848a7d2c, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 443d4e0e-3f2a-42c9-ad16-6c7537ff8c60)

In [ ]:
fact_df_final.show()

StatementMeta(, 9bad9778-0cdd-48f9-8968-ca1f005c3be5, 55, Finished, Available, Finished)

+-------+-----------+-----------+-------------+---------------+-------+--------------------+-------------+
|fact_id|employee_id|location_id|department_id|total_employees|avg_age| gender_distribution|turnover_rate|
+-------+-----------+-----------+-------------+---------------+-------+--------------------+-------------+
|      1|        101|         22|            3|             14|   38.6|Male: 0.0%, Femal...|         7.14|
|      2|        102|         57|            3|              2|   42.0|Male: 0.0%, Femal...|          0.0|
|      3|        103|         55|            7|              1|   55.0|Male: 0.0%, Femal...|          0.0|
|      4|        104|          2|           10|              2|   43.5|Male: 0.0%, Femal...|          0.0|
|      5|        105|         15|            3|             17|   44.6|Male: 0.0%, Femal...|        17.65|
|      6|        106|         49|            5|             10|   39.7|Male: 0.0%, Femal...|         20.0|
|      7|        107|         40|    

In [ ]:
fact_df.write.format("delta") \
    .mode("overwrite") \
    .save(gold_lh_tpath + "//" + "FactEmployee")

StatementMeta(, 7d6c9461-c616-465c-b378-855d848a7d2c, 18, Finished, Available, Finished)